In [1]:
import tensorflow as tf
import pygame as pg
import numpy as np
import sys

#LINEAR

def predict(x):
    tfx = tf.Variable(x, dtype='float32')
    return tf.add(
        tf.multiply(tfx, m), 
        b)

def normPos(pos, w, h):
    w2 = w / 2
    h2 = h / 2
    if len(pos) != 2:
        raise Exception("Cannot normalize coordinates to 2D")
    normPos = []
    normPos.append((pos[0] - w2) / w2)
    normPos.append((pos[1] - h2) / h2)
    return normPos

def denormPos(normPos, w, h):
    w2 = w / 2
    h2 = h / 2
    if len(normPos) != 2:
        raise Exception("Cannot denormalize coordinates to 2D")
    pos = []
    pos.append(int((normPos[0] * w2) + w2))
    pos.append(int((normPos[1] * h2) + w2))
    return pos

w = 600
h = 600
xPos = []
yPos = []

m = tf.Variable(np.random.uniform(-1, 1))
b = tf.Variable(np.random.uniform(-1, 1))

lr = 0.1
opt = tf.keras.optimizers.SGD(learning_rate=lr)

pg.init()
window = pg.display.set_mode((w, h))

while True:
    window.fill((255, 255, 255))
    for event in pg.event.get():
        if event.type == pg.QUIT:
            pg.quit()
            sys.exit()
        elif event.type == pg.MOUSEBUTTONDOWN:
            # Left mouse button
            if event.button == 1:
                pos = normPos(pg.mouse.get_pos(), w, h)
                xPos.append(pos[0])
                yPos.append(pos[1])
        elif event.type == pg.KEYDOWN:
            if event.key == pg.K_SPACE:
                xPos.clear()
                yPos.clear()
    for i in range(len(xPos)):
        pg.draw.circle(window, (0, 0, 0), denormPos([xPos[i], yPos[i]], w, h), 5)
    
    pg.draw.aaline(window, (255, 0, 0), denormPos([-1, predict(-1)], w, h), denormPos([1,  predict(1)], w, h))

    with tf.GradientTape() as tape:
        loss = tf.keras.losses.mse(predict(xPos), yPos)
    opt.minimize(loss, [m, b], tape=tape)
    pg.display.update()

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
import tensorflow as tf
import pygame as pg
import numpy as np
import sys

#POLYNOMIAL

def predict(x):
    tfx = tf.Variable(x, dtype='float32')

    return tf.add(tf.add(tf.add(
        tf.multiply(tf.pow(tfx, 3), a), 
        tf.multiply(tf.square(tfx), b)), 
        tf.multiply(tfx, c)), 
        d)

def normPos(pos, w, h):
    w2 = w / 2
    h2 = h / 2
    if len(pos) != 2:
        raise Exception("Cannot normalize coordinates to 2D")
    normPos = []
    normPos.append((pos[0] - w2) / w2)
    normPos.append((pos[1] - h2) / h2)
    return normPos

def denormPos(normPos, w, h):
    w2 = w / 2
    h2 = h / 2
    if len(normPos) != 2:
        raise Exception("Cannot denormalize coordinates to 2D")
    pos = []
    pos.append(int((normPos[0] * w2) + w2))
    pos.append(int((normPos[1] * h2) + w2))
    return pos

def curvePoints(num, w, h):
    lines = []
    for i in np.linspace(-1, 1, num):
        lines.append(denormPos([i, predict(i)], w, h))
    return lines

def equationString(A, B, C, D):
    return "y = " + str(
        round(A, 2)) + "x^3 + " + str(
        round(B, 2)) + "x^2 + " + str(
        round(C, 2)) + "x + " + str(
        round(D, 2))

w = 600
h = 600
xPos = []
yPos = []

a = tf.Variable(np.random.uniform(-1, 1))
b = tf.Variable(np.random.uniform(-1, 1))
c = tf.Variable(np.random.uniform(-1, 1))
d = tf.Variable(np.random.uniform(-1, 1))

lr = 0.05
opt = tf.keras.optimizers.Adam(learning_rate=lr)

pg.init()
pg.font.init()
window = pg.display.set_mode((w, h))

while True:
    window.fill((255, 255, 255))
    for event in pg.event.get():
        if event.type == pg.QUIT:
            pg.quit()
            sys.exit()
        elif event.type == pg.MOUSEBUTTONDOWN:
            # Left mouse button
            if event.button == 1:
                pos = normPos(pg.mouse.get_pos(), w, h)
                xPos.append(pos[0])
                yPos.append(pos[1])
        elif event.type == pg.KEYDOWN:
            if event.key == pg.K_SPACE:
                xPos.clear()
                yPos.clear()
    for i in range(len(xPos)):
        pg.draw.circle(window, (0, 0, 0), denormPos([xPos[i], yPos[i]], w, h), 5)
    
    pg.draw.aalines(window, (0, 0, 255), False, curvePoints(25, w, h), 5)
    font = pg.font.SysFont(None, 24)
    trainCount = font.render(equationString(a.numpy(), b.numpy(), c.numpy(), d.numpy()), True, (255, 0, 0))
    center = trainCount.get_rect(center=(w / 2, h - 24))
    window.blit(trainCount, center)
    
    with tf.GradientTape() as tape:
        loss = tf.keras.losses.mse(predict(xPos), yPos)
    opt.minimize(loss, [a, b, c, d], tape=tape)
    pg.display.update()

ModuleNotFoundError: No module named 'tensorflow'